Based on the TESS beginner_how_to_use_lc notebook

In [ ]:
%matplotlib inline
from astropy.io import fits
from astropy import coordinates as c
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql

In [ ]:
data_dir = 'data/light_curves/TESS-WASP17/'

obs1_base = "tess2021118034608-s0038-0000000066818296-0209-s"
obs2_base = "tess2019140104343-s0012-0000000066818296-0144-s"

obs1_lc = f"{data_dir}/{obs1_base}/{obs1_base}_lc.fits"
obs2_lc = f"{data_dir}/{obs2_base}/{obs2_base}_lc.fits"
obs_files = [obs1_lc, obs2_lc]

### Data exploration

In [ ]:
with fits.open(obs2_lc, mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    sap_fluxes = hdulist[1].data['SAP_FLUX']
    pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']
    pdcsap_err = hdulist[1].data['PDCSAP_FLUX_ERR']
    quality = hdulist[1].data['QUALITY']

In [ ]:
# Define the epoch of primary transit in TBJD.  Our timestamps are also already in TBJD.
t0 = 1327.520678

# Start figure and axis.
fig, ax = plt.subplots()

# Plot the timeseries in black circles.
ax.plot(tess_bjds, pdcsap_fluxes, 'ko')

# Center the x-axis on where we expect a transit to be (time = T0), and set
# the x-axis range within +/- 1 day of T0.
ax.set_xlim(t0 - 1.0, t0 + 1.0)

# Overplot a red vertical line that should be where the transit occurs.
ax.axvline(x=t0, color="red")

# Let's label the axes and define a title for the figure.
fig.suptitle("WASP-126 b Light Curve - Sector 1")
ax.set_ylabel("PDCSAP Flux (e-/s)")
ax.set_xlabel("Time (TBJD)")

# Adjust the left margin so the y-axis label shows up.
plt.subplots_adjust(left=0.15)
plt.show()

In [ ]:
bad_bits = np.array([1,2,3,4,5,6,8,10,12])
value = 0
for v in bad_bits:
    value = value + 2**(v-1)
    
bad_data = np.bitwise_and(quality, value) >= 1 
bad_data[np.isnan(pdcsap_fluxes)] = True

In [ ]:
#Same plot as before with flagged data removed.
fig = plt.figure(figsize = (14,5))
fig.add_subplot(211)
plt.plot(tess_bjds[~bad_data], sap_fluxes[~bad_data], '.', label = 'SAP')
plt.legend(loc = 'lower left')

fig.add_subplot(212)
plt.plot(tess_bjds[~bad_data], pdcsap_fluxes[~bad_data], '.', label = 'PDC')
plt.legend(loc = 'lower left')
plt.xlabel('TIME (BTJD)')

Simple normalization. Note that there are apparently still NaNs in the pdcsap fluxes after removing quality-flagged pixels, so `np.nanmean` is required

In [ ]:
#Same plot as before with flagged data removed.
fig = plt.figure(figsize = (14,5))
fig.add_subplot(211)
plt.plot(tess_bjds[~bad_data], 
         sap_fluxes[~bad_data]/np.nanmean(sap_fluxes[~bad_data]), 
         '.', label = 'SAP')
plt.legend(loc = 'lower left')

fig.add_subplot(212)
# 2021 data
#non_transit = np.where(pdcsap_fluxes[~bad_data] > 6200)
# 2019 data
non_transit = np.where(pdcsap_fluxes[~bad_data] > 6325)
plt.plot(tess_bjds[~bad_data], 
         pdcsap_fluxes[~bad_data]/np.nanmean(pdcsap_fluxes[~bad_data][non_transit]), 
         '.', label = 'PDC')
plt.legend(loc = 'lower left')
plt.xlabel('TIME (BTJD)')

Let's try binning and see what those look like...

Actually, this will take more time to figure out how to do intelligently than I want to spend right now. Juliet certainly doesn't need the data to be binned, and the transits are pretty obvious in the plot above anyway!

In [ ]:
print(tess_bjds.shape)
print(tess_bjds[~bad_data].shape)

In [ ]:
good_times = tess_bjds[~bad_data] 
normalized_flux = (pdcsap_fluxes[~bad_data] /
                   np.nanmean(pdcsap_fluxes[~bad_data][non_transit]))
normalized_errs = (pdcsap_err[~bad_data] / pdcsap_fluxes[~bad_data])*normalized_flux

In [ ]:
print(normalized_flux[1000:1010])

In [ ]:
print(normalized_errs[1000:1010])

In [ ]:
print(np.nanmean(pdcsap_fluxes[~bad_data]))
print(np.nanstd(pdcsap_fluxes[~bad_data]))
print(np.nanmean(pdcsap_fluxes[~bad_data]) - 2*np.nanstd(pdcsap_fluxes[~bad_data]))

### Insert the TESS data into the SHEL database

In [ ]:
# initialize database connection
conn = sql.connect("shel_database.sqlite")
cur = conn.cursor()

In [ ]:
# For reprocessing
# cur.execute("delete from light_curves where target_id=1")

In [ ]:
# Insert target/instrument information into the database
sky = c.SkyCoord(ra="15h 59m 50.939s", dec="-28deg 03m 42.48s")

stmt = f"INSERT INTO targets (name, ra, dec) VALUES ('WASP-17', {sky.ra.degree}, {sky.dec.degree})"
cur.execute(stmt)

stmt = f"INSERT INTO instruments (name) VALUES ('TESS')"
cur.execute(stmt)

In [ ]:
cur.execute("SELECT * from instruments")
print(cur.fetchall())

cur.execute("SELECT * from targets")
print(cur.fetchall())

In [ ]:
# Calculate normalized fluxes and write to database for both files

for fname in obs_files:
    with fits.open(fname, mode="readonly") as hdulist:
        tess_bjds = hdulist[1].data['TIME']
        sap_fluxes = hdulist[1].data['SAP_FLUX']
        pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']
        pdcsap_err = hdulist[1].data['PDCSAP_FLUX_ERR']
        quality = hdulist[1].data['QUALITY']
    
        # Reject bad quality flags
        bad_bits = np.array([1,2,3,4,5,6,8,10,12])
        value = 0
        for v in bad_bits:
            value = value + 2**(v-1)
        bad_data = np.bitwise_and(quality, value) >= 1
        bad_data[np.isnan(pdcsap_fluxes)] = True
    
        # Reject anything below this to get non-transit mean
        min_threshold = (np.nanmean(pdcsap_fluxes[~bad_data]) - 
                         2*np.nanstd(pdcsap_fluxes[~bad_data]))
        non_transit = np.where(pdcsap_fluxes[~bad_data] > min_threshold)
    
        # Convert the times to unsubtracted BJD
        good_times = tess_bjds[~bad_data] + 2457000
        normalized_flux = (pdcsap_fluxes[~bad_data] /
                           np.nanmean(pdcsap_fluxes[~bad_data][non_transit]))
        normalized_errs = normalized_flux*(pdcsap_err[~bad_data] / 
                                           pdcsap_fluxes[~bad_data])
        for i in range(len(good_times)):
            stmt = ("INSERT INTO light_curves (target_id, instrument, bjd, flux, flux_err)"
                    f" values (1, 1, {good_times[i]}, {normalized_flux[i]}, {normalized_errs[i]})")
            cur.execute(stmt)

In [ ]:
cur.execute("select count(*) from light_curves").fetchall()

In [ ]:
# Commit our database changes
conn.commit()
cur.close()
conn.close()

### Utility function version

In [ ]:
from utils import ingest_tess_data

In [ ]:
# Run with debug=True first to check that values in insert statements make sense

datadir = 'data/light_curves/TESS-HAT-P-11/MAST_2022-03-23T1730/TESS/'

ingest_tess_data(datadir, "HAT-P-11", debug=True)